In [1]:
import torch
import matplotlib.pyplot as plt
import random

# Callbacks

## Callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description='Click me')

In [4]:
w

Button(description='Click me', style=ButtonStyle())

In [5]:
def f(o):
  print('hi')

In [6]:
w.on_click(f)

## Creating your own callback

In [7]:
from time import sleep

In [8]:
def slow_calculation():
  res = 0
  for i in range(5):
    res += i*i
    sleep(1)
  return res

In [9]:
slow_calculation()

30

In [10]:
def slow_calculation(cb=None):
  res = 0
  for i in range(5):
    res += i*i
    sleep(1)
    if cb:
      cb(i)
  return res

In [11]:
def show_progress(epoch):
  print(f"Awesome! We've finished epoch {epoch} ")

In [12]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0 
Awesome! We've finished epoch 1 
Awesome! We've finished epoch 2 
Awesome! We've finished epoch 3 
Awesome! We've finished epoch 4 


30

## Lambdas and partials

In [13]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [14]:
def show_progress(exclamation, epoch):
  print(f"{exclamation} We've finished epoch {epoch}")

In [15]:
slow_calculation(lambda o: show_progress('Ok I guess', o))

Ok I guess We've finished epoch 0
Ok I guess We've finished epoch 1
Ok I guess We've finished epoch 2
Ok I guess We've finished epoch 3
Ok I guess We've finished epoch 4


30

In [16]:
def make_show_progress(exclamation):
  _inner = lambda epoch: print(f"{exclamation}! We've finished epoch {epoch}")
  return _inner

In [17]:
slow_calculation(make_show_progress('Nice'))

Nice! We've finished epoch 0
Nice! We've finished epoch 1
Nice! We've finished epoch 2
Nice! We've finished epoch 3
Nice! We've finished epoch 4


30

In [18]:
from functools import partial

In [19]:
slow_calculation(partial(show_progress, "Ok I guess"))

Ok I guess We've finished epoch 0
Ok I guess We've finished epoch 1
Ok I guess We've finished epoch 2
Ok I guess We've finished epoch 3
Ok I guess We've finished epoch 4


30

In [20]:
f2 = partial(show_progress, 'Ok I guess')

In [21]:
f2(2)

Ok I guess We've finished epoch 2


## Callbacks as callable classes

In [22]:
class ProgressShowingCallback():
  def __init__(self, exclamation='Awesome'):
    self.exclamation = exclamation
  def __call__(self, epoch):
    print(f"{self.exclamation}! We've finished epoch {epoch}")

In [23]:
cb = ProgressShowingCallback('Just super')

In [24]:
slow_calculation(cb)

Just super! We've finished epoch 0
Just super! We've finished epoch 1
Just super! We've finished epoch 2
Just super! We've finished epoch 3
Just super! We've finished epoch 4


30

## Multiple callback funcs; *args and **kwargs

In [25]:
def f(*args, **kwargs):
  print(f"args: {args}; kwargs {kwargs}")

In [26]:
f(3, 'a', thing1='hello')

args: (3, 'a'); kwargs {'thing1': 'hello'}


In [27]:
def slow_calculation(cb=None):
  res = 0
  for i in range(5):
    if cb:
      cb.before_calc(i)
    res += i*i
    sleep(1)
    if cb:
      cb.after_calc(i, val=res)
  return res

In [28]:
class PrintStepCallback():
  def __init__(self):
    pass
  def before_calc(self, *args, **kwargs):
    print(f"About to start")
  def after_calc(self, *args, **kwargs):
    print(f"Done step")

In [29]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [30]:
class PrintStatusCallback():
  def __init__(self):
    pass
  def before_calc(self, epoch, **kwargs):
    print(f"About to start: {epoch}")
  def after_calc(self, epoch, val, **kwargs):
    print(f"After {epoch}: {val}")

In [31]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

## __dunder__ things

In [32]:
class SloppyAdder():
  def __init__(self, o):
    self.o = o
  def __add__(self, b):
    return SloppyAdder(self.o + b.o + 0.01)
  def __repr__(self):
    return str(self.o)

In [33]:
a = SloppyAdder(1)
b = SloppyAdder(2)

a + b

3.01

In [35]:
class A:
  a,b = 1,2

In [36]:
a = A()

In [37]:
a.b

2

In [40]:
getattr(a, 'b' if  random.random()>0.5 else 'a')

1

In [41]:
class B:
  a,b = 1,2
  def __getattr__(self, k):
    if k[0] =='_': 
      raise AttributeError(k)
    return f'Hello from {k}'

In [42]:
b = B()

In [43]:
b.a

1

In [44]:
b.foo

'Hello from foo'